In [1]:
import pandas as pd
import mysql.connector

# 1. MySQL 연결
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="분석실습"
)
cursor = conn.cursor()


In [5]:
mysql.connector.__version__

'9.3.0'

In [6]:
# 2. 파일에서 데이터프레임으로 읽기 (탭 구분자라고 가정)
df_customer = pd.read_csv("mig/customer.txt", sep="\t", dtype=str,  encoding='cp949')
df_customer.shape
#(5369, 8)

(5369, 8)

In [7]:
df_sales = pd.read_csv("mig/sales.txt", sep="\t", dtype=str,  encoding='cp949')
df_sales.shape

(541909, 8)

In [8]:
# 3. CUSTOMER INSERT
for row in df_customer.itertuples(index=False, name=None):
    cursor.execute("""
        INSERT INTO CUSTOMER (mem_no, last_name, first_name, gd, birth_dt, entr_dt, grade, sign_up_ch)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, row)
    
conn.commit()  # 변경사항 저장

cursor.execute("SELECT COUNT(*) FROM CUSTOMER")
print(f"총 {cursor.fetchone()[0]}건 삽입 완료")

총 5369건 삽입 완료


In [9]:
# 4. SALES 데이터전처리
df_sales = df_sales.dropna(subset=["CustomerID"])
df_sales["CustomerID"] = df_sales["CustomerID"].astype(int)

# 날짜 및 숫자 변환
df_sales["Quantity"] = df_sales["Quantity"].astype(int)
df_sales["UnitPrice"] = df_sales["UnitPrice"].astype(float)
df_sales["CustomerID"] = df_sales["CustomerID"].astype(int)

# DATETIME 변환
df_sales["InvoiceDate"] = pd.to_datetime(df_sales["InvoiceDate"], errors='coerce')
df_sales["InvoiceDate"] = df_sales["InvoiceDate"].dt.strftime('%Y-%m-%d %H:%M:%S')
df_sales.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [10]:
# 5. SALES INSERT
df_sales = df_sales.astype(object)
rows = [tuple(x) for x in df_sales.to_records(index=False)]
cursor.executemany("""
    INSERT INTO SALES (InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
""", rows)

conn.commit()

cursor.execute("SELECT COUNT(*) FROM SALES")
print(f"총 {cursor.fetchone()[0]}건 삽입 완료")

총 406829건 삽입 완료


In [3]:
# 6. 종료
cursor.close()
conn.close()
print("데이터 삽입 완료")


데이터 삽입 완료


# 매출 트렌드

In [4]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="분석실습"
)
cursor = conn.cursor()


cursor.execute("""
                SELECT invoicedate
               , SUM(unitprice*quantity) AS 매출액
               , SUM(quantity) AS 주문수량
               , COUNT(DISTINCT invoiceno) AS 주문건수
               , COUNT(DISTINCT customerid) AS 주문고객수
               FROM sales
               GROUP BY invoicedate
               ORDER BY invoicedate
""")


df = pd.DataFrame(cursor.fetchall())
print(df.head(5))


                    0       1    2  3  4
0 2010-12-01 08:26:00  139.12   40  1  1
1 2010-12-01 08:28:00   22.20   12  1  1
2 2010-12-01 08:34:00  348.78   98  2  1
3 2010-12-01 08:35:00   17.85    3  1  1
4 2010-12-01 08:45:00  855.86  449  1  1


## 국가별 매출 현황

In [5]:
cursor.execute("""
                SELECT country
               , SUM(unitprice*quantity) AS 매출액
               , SUM(quantity) AS 주문수량
               , COUNT(DISTINCT invoiceno) AS 주문건수
               , COUNT(DISTINCT customerid) AS 주문고객수
               FROM sales
               GROUP BY country
""")


df = pd.DataFrame(cursor.fetchall())
print(df.head(5))

           0          1      2    3   4
0  Australia  137077.27  83653   69   9
1    Austria   10154.32   4827   19  11
2    Bahrain     548.40    260    2   2
3    Belgium   40910.96  23152  119  25
4     Brazil    1143.60    356    1   1


## 국가, 제품별

In [6]:
cursor.execute("""
                SELECT country
               , stockcode
               , SUM(unitprice*quantity) AS 매출액
               , SUM(quantity) AS 주문수량
               , COUNT(DISTINCT invoiceno) AS 주문건수
               , COUNT(DISTINCT customerid) AS 주문고객수
               FROM sales
               GROUP BY country, stockcode
""")

col = ["country", "stockcode", "매출액", "주문수량", "주문건수", "주문고객수"]
df = pd.DataFrame(cursor.fetchall(), columns=col)
print(df.head(5))

     country stockcode     매출액 주문수량  주문건수  주문고객수
0  Australia     15036  432.00  600     1      1
1  Australia   15056BL   17.85    3     1      1
2  Australia    16161P  136.00  400     1      1
3  Australia    16169E   10.50   25     1      1
4  Australia     20665   17.70    6     1      1


## 특정 제품의 매출 지표

In [7]:
cursor.execute("""
                SELECT SUM(unitprice*quantity) AS 매출액
               , SUM(quantity) AS 주문수량
               , COUNT(DISTINCT invoiceno) AS 주문건수
               , COUNT(DISTINCT customerid) AS 주문고객수
               FROM sales
               WHERE stockcode = 21615
""")

col = ["매출액", "주문수량", "주문건수", "주문고객수"]
df = pd.DataFrame(cursor.fetchall(), columns=col)
print(df)

       매출액 주문수량  주문건수  주문고객수
0  1195.00  840   100     86


In [8]:
cursor.execute("""
                SELECT invoicedate
               , SUM(unitprice*quantity) AS 매출액
               , SUM(quantity) AS 주문수량
               , COUNT(DISTINCT invoiceno) AS 주문건수
               , COUNT(DISTINCT customerid) AS 주문고객수
               FROM sales
               WHERE stockcode IN ('21615', '21731')
               GROUP BY invoicedate
""")

col = ["invoicedate", "매출액", "주문수량", "주문건수", "주문고객수"]
df = pd.DataFrame(cursor.fetchall(), columns=col)
print(df.head(5))

          invoicedate     매출액 주문수량  주문건수  주문고객수
0 2010-12-01 08:45:00   39.60   24     1      1
1 2010-12-01 09:41:00    3.30    2     1      1
2 2010-12-01 09:58:00  540.00  432     1      1
3 2010-12-01 11:49:00    8.25    5     1      1
4 2010-12-01 13:17:00    3.30    2     1      1


## 이벤트 효과 분석

In [9]:
cursor.execute("""
                SELECT CASE WHEN invoicedate BETWEEN '2011-09-10' AND '2011-09-25' THEN '이벤트 기간'
                            WHEN invoicedate BETWEEN '2011-08-10' AND '2011-08-25' THEN '이벤트 비교기간'
                            END AS 기간구분 
               , SUM(unitprice*quantity) AS 매출액
               , SUM(quantity) AS 주문수량
               , COUNT(DISTINCT invoiceno) AS 주문건수
               , COUNT(DISTINCT customerid) AS 주문고객수
               FROM sales
               WHERE invoicedate BETWEEN '2011-09-10' AND '2011-09-25' 
                    OR invoicedate BETWEEN '2011-08-10' AND '2011-08-25'
               GROUP BY CASE WHEN invoicedate BETWEEN '2011-09-10' AND '2011-09-25' THEN '이벤트 기간'
                            WHEN invoicedate BETWEEN '2011-08-10' AND '2011-08-25' THEN '이벤트 비교기간'
                            END
""")

col = ["기간구분", "매출액", "주문수량", "주문건수", "주문고객수"]
df = pd.DataFrame(cursor.fetchall(), columns=col)
print(df.head(5))

       기간구분        매출액    주문수량  주문건수  주문고객수
0    이벤트 기간  509349.88  277034   966    705
1  이벤트 비교기간  348331.09  215201   802    603


## 이벤트 제품 효과 분석 (시기별)

In [10]:
cursor.execute("""
                SELECT CASE WHEN invoicedate BETWEEN '2011-09-10' AND '2011-09-25' THEN '이벤트 기간'
                            WHEN invoicedate BETWEEN '2011-08-10' AND '2011-08-25' THEN '이벤트 비교기간'
                            END AS 기간구분 
               , SUM(unitprice*quantity) AS 매출액
               , SUM(quantity) AS 주문수량
               , COUNT(DISTINCT invoiceno) AS 주문건수
               , COUNT(DISTINCT customerid) AS 주문고객수
               FROM sales
               WHERE (invoicedate BETWEEN '2011-09-10' AND '2011-09-25' 
                    OR invoicedate BETWEEN '2011-08-10' AND '2011-08-25')
                    AND stockcode IN ('17012A', '17012C', '17021', '17084BN')
               GROUP BY CASE WHEN invoicedate BETWEEN '2011-09-10' AND '2011-09-25' THEN '이벤트 기간'
                            WHEN invoicedate BETWEEN '2011-08-10' AND '2011-08-25' THEN '이벤트 비교기간'
                            END
""")

col = ["기간구분", "매출액", "주문수량", "주문건수", "주문고객수"]
df = pd.DataFrame(cursor.fetchall(), columns=col)
print(df.head(5))

       기간구분    매출액 주문수량  주문건수  주문고객수
0    이벤트 기간  58.15  115     8      8
1  이벤트 비교기간  13.35   28     5      5


## 특정 제품 구매 고객 정보

In [11]:
cursor.execute("""
               SELECT s.customerid
               , c.customer_name
               , c.gd
               , c.birth_dt
               , c.grade
               , c.sign_up_ch
               FROM (
               SELECT DISTINCT customerid 
               FROM sales 
               WHERE stockcode IN ('21730', '21615')
               AND invoicedate BETWEEN '2010-12-01' AND '2010-12-10'
               ) as s
               JOIN (
               SELECT mem_no
               , CONCAT(last_name, first_name) AS customer_name
               , gd
               , birth_dt
               , entr_dt
               , grade
               , sign_up_ch
               FROM customer
               ) c
               on s.customerid = c.mem_no
""")

df = pd.DataFrame(cursor.fetchall())
print(df.head(5))

       0    1  2           3       4   5
0  17850  임은성  M  1995-02-01  BRONZE  02
1  16883  윤제윤  M  1979-09-22  BRONZE  03
2  17062  유강산  M  1975-01-17  BRONZE  04
3  14443  이미연  F  1986-10-02  BRONZE  02
4  14344  김화영  F  1987-07-02  BRONZE  02


## 미구매 고객 정보 출력

In [12]:
cursor.execute("""
               SELECT CASE WHEN s.CustomerID IS NULL THEN c.mem_no END non_purchaser
               , c.mem_no
               , c.last_name
               , c.first_name
               , s.InvoiceNo
               , s.stockcode
               , s.quantity
               , s.invoicedate
               , s.unitprice
               , s.customerid
               FROM customer c
               JOIN sales s
               ON c.mem_no = s.customerid
""")

df = pd.DataFrame(cursor.fetchall())
print(df.head(5))

      0      1  2   3       4       5  6                   7     8      9
0  None  17850  임  은성  536365  85123A  6 2010-12-01 08:26:00  2.55  17850
1  None  17850  임  은성  536365   71053  6 2010-12-01 08:26:00  3.39  17850
2  None  17850  임  은성  536365  84406B  8 2010-12-01 08:26:00  2.75  17850
3  None  17850  임  은성  536365  84029G  6 2010-12-01 08:26:00  3.39  17850
4  None  17850  임  은성  536365  84029E  6 2010-12-01 08:26:00  3.39  17850


## 전체 고객수, 미구매 고객수

In [13]:
cursor.execute("""
               SELECT COUNT(DISTINCT CASE WHEN s.CustomerID IS NULL THEN c.mem_no END ) AS non_purchaser
               , COUNT(DISTINCT c.mem_no) AS total_customer
               FROM customer c
               JOIN sales s
               ON c.mem_no = s.CustomerID
""")

df = pd.DataFrame(cursor.fetchall(), columns=["non_purchaser", "total_customer"])
print(df)

   non_purchaser  total_customer
0              0            4372


## 매출 평균 지표

In [14]:
cursor.execute("""
                SELECT SUM(unitprice*quantity) AS 매출액
               , SUM(quantity) AS 주문수량
               , COUNT(DISTINCT invoiceno) AS 주문건수
               , COUNT(DISTINCT customerid) AS 주문고객수
               , SUM(unitprice*quantity) / COUNT(DISTINCT invoiceno) AS ATV
               , SUM(unitprice*quantity) / COUNT(DISTINCT customerid) AS AMV
               , COUNT(DISTINCT invoiceno) * 1.00 / COUNT(DISTINCT customerid) AS  AvgFrq
               , SUM(quantity) * 1.00 / COUNT(DISTINCT invoiceno) AS AvgUnits
               FROM sales
""")

col = ["매출액", "주문수량", "주문건수", "주문고객수", "ATV", "AMV", "AvgFrq", "AvgUnits"]
df = pd.DataFrame(cursor.fetchall(), columns=col)
print(df)

          매출액     주문수량   주문건수  주문고객수         ATV          AMV    AvgFrq  \
0  8300065.81  4906888  22190   4372  374.045327  1898.459700  5.075480   

     AvgUnits  
0  221.130599  


## 매출 평균 지표 (연도, 월별)

In [15]:
cursor.execute("""
               SELECT  YEAR(invoicedate)
               , MONTH(invoicedate)
               , (unitprice*quantity) AS 매출액
               , SUM(quantity) AS 주문수량
               , COUNT(DISTINCT invoiceno) AS 주문건수
               , COUNT(DISTINCT customerid) AS 주문고객수
               , SUM(unitprice*quantity) / COUNT(DISTINCT invoiceno) AS ATV
               , SUM(unitprice*quantity) / COUNT(DISTINCT customerid) AS AMV
               , COUNT(DISTINCT invoiceno) * 1.00 / COUNT(DISTINCT customerid) AS  AvgFrq
               , SUM(quantity) * 1.00 / COUNT(DISTINCT invoiceno) AS AvgUnits
               FROM sales
               GROUP BY YEAR(invoicedate)
               , MONTH(invoicedate)
               ORDER BY 1, 2
""")

df = pd.DataFrame(cursor.fetchall())
print(df.head(2))

ProgrammingError: 1055 (42000): Expression #3 of SELECT list is not in GROUP BY clause and contains nonaggregated column '분석실습.sales.UnitPrice' which is not functionally dependent on columns in GROUP BY clause; this is incompatible with sql_mode=only_full_group_by

## 특정연도 베스트 셀링 상품 확인

In [20]:
cursor.execute("""
               SELECT stockcode
               , description
               , SUM(quantity) as qty
               FROM sales
               WHERE YEAR(invoicedate) = '2011'
               GROUP BY stockcode
               , description
               ORDER BY qty DESC
""")

df = pd.DataFrame(cursor.fetchall())
print(df.head(10))

        0                                   1      2
0   84077   WORLD WAR 2 GLIDERS ASSTD DESIGNS  48076
1  85099B             JUMBO BAG RED RETROSPOT  43036
2   84879       ASSORTED COLOUR BIRD ORNAMENT  33047
3  85123A  WHITE HANGING HEART T-LIGHT HOLDER  31064
4   22197                      POPCORN HOLDER  30504
5   21212     PACK OF 72 RETROSPOT CAKE CASES  29867
6   23084                  RABBIT NIGHT LIGHT  27094
7   22492             MINI PAINT SET VINTAGE   23180
8   22616          PACK OF 12 LONDON TISSUES   22665
9   21977  PACK OF 60 PINK PAISLEY CAKE CASES  22531


## 국가별 베스트상품

In [24]:
cursor.execute("""
               SELECT ROW_NUMBER() OVER (PARTITION BY country ORDER BY qty DESC) AS rnk
                    , country
                    , stockcode
                    , description
                    , qty
               FROM (
                    SELECT country
                    , stockcode
                    , description
                    , SUM(quantity) as qty
                    FROM sales
                    GROUP BY country
                    , stockcode
                    , description
               ) a
               ORDER BY 2, 1
""")

df = pd.DataFrame(cursor.fetchall())
print(df.head(10))

    0          1      2                              3     4
0   1  Australia  22492        MINI PAINT SET VINTAGE   2916
1   2  Australia  23084             RABBIT NIGHT LIGHT  1884
2   3  Australia  21915         RED  HARMONICA IN BOX   1704
3   4  Australia  21731  RED TOADSTOOL LED NIGHT LIGHT  1344
4   5  Australia  22630           DOLLY GIRL LUNCH BOX  1024
5   6  Australia  22969   HOMEMADE JAM SCENTED CANDLES   994
6   7  Australia  22629            SPACEBOY LUNCH BOX    960
7   8  Australia  22544           MINI JIGSAW SPACEBOY   720
8   9  Australia  21914         BLUE HARMONICA IN BOX    720
9  10  Australia  22620    4 TRADITIONAL SPINNING TOPS   700


## 20대 여성 고객의 베스트셀링 상품 확인

In [2]:
cursor.execute("""
               SELECT *
               FROM (
               SELECT ROW_NUMBER() OVER (ORDER BY qty DESC) AS rnk
                    , stockcode
                    , description
                    , qty
               FROM (
               SELECT stockcode
               , description
               , SUM(quantity) as qty
               FROM sales s
               JOIN customer c
               ON s.customerid = c.mem_no
               WHERE c.gd = 'F'
               AND 2023-year(c.birth_dt) BETWEEN '20' AND '29'
               GROUP BY stockcode
               , description
               ) a
               ) aa
               WHERE rnk <= 10
""")

df = pd.DataFrame(cursor.fetchall())
print(df.head(10))

    0       1                                  2     3
0   1   22492            MINI PAINT SET VINTAGE   3527
1   2   22151          PLACE SETTING WHITE HEART  3150
2   3  85099B            JUMBO BAG RED RETROSPOT  2444
3   4   23084                 RABBIT NIGHT LIGHT  2424
4   5   21915             RED  HARMONICA IN BOX   2421
5   6   22178    VICTORIAN GLASS HANGING T-LIGHT  1792
6   7   79321                      CHILLI LIGHTS  1786
7   8   23167     SMALL CERAMIC TOP STORAGE JAR   1786
8   9   84077  WORLD WAR 2 GLIDERS ASSTD DESIGNS  1779
9  10   21137           BLACK RECORD COVER FRAME  1598


## 특정 제품과 함께 가장 많이 구매한 제품

In [3]:
cursor.execute("""
               SELECT DISTINCT invoiceno
               FROM sales
               WHERE stockcode = '20725'
               """)


df = pd.DataFrame(cursor.fetchall())
print(df.head(10))

        0
0  536378
1  536401
2  536412
3  536577
4  536769
5  536845
6  536863
7  536944
8  536974
9  537047


In [8]:
cursor.execute("""
               SELECT s.stockcode
               , s.description
               , SUM(quantity) as qty
               FROM sales s
               INNER
               JOIN (
               SELECT DISTINCT invoiceno
               FROM sales
               WHERE stockcode = '20725'
               ) i
               ON s.invoiceno = i.invoiceno
               WHERE s.stockcode <> '20725'
               GROUP BY s.stockcode
               , s.description
               ORDER BY qty DESC
               """)


df = pd.DataFrame(cursor.fetchall())
print(df.head(10))

        0                                1     2
0  85099B          JUMBO BAG RED RETROSPOT  8510
1   22384          LUNCH BAG PINK POLKADOT  5925
2   21212  PACK OF 72 RETROSPOT CAKE CASES  5422
3   20727          LUNCH BAG  BLACK SKULL.  5305
4   20728              LUNCH BAG CARS BLUE  5009
5   20726               LUNCH BAG WOODLAND  4986
6   22383           LUNCH BAG SUKI DESIGN   4874
7   23206           LUNCH BAG APPLE DESIGN  4775
8   22382       LUNCH BAG SPACEBOY DESIGN   4391
9   84879    ASSORTED COLOUR BIRD ORNAMENT  3734


## LUNCH 제외

In [9]:
cursor.execute("""
               SELECT s.stockcode
               , s.description
               , SUM(quantity) as qty
               FROM sales s
               INNER
               JOIN (
               SELECT DISTINCT invoiceno
               FROM sales
               WHERE stockcode = '20725'
               ) i
               ON s.invoiceno = i.invoiceno
               WHERE s.stockcode <> '20725'
               AND s.description NOT LIKE '%LUNCH%'
               GROUP BY s.stockcode
               , s.description
               ORDER BY qty DESC
               """)


df = pd.DataFrame(cursor.fetchall())
print(df.head(10))

        0                                  1     2
0  85099B            JUMBO BAG RED RETROSPOT  8510
1   21212    PACK OF 72 RETROSPOT CAKE CASES  5422
2   84879      ASSORTED COLOUR BIRD ORNAMENT  3734
3   22386            JUMBO BAG PINK POLKADOT  3655
4   20724        RED RETROSPOT CHARLOTTE BAG  3583
5   23084                 RABBIT NIGHT LIGHT  3544
6   21931             JUMBO STORAGE BAG SUKI  3163
7   22492            MINI PAINT SET VINTAGE   2958
8   84077  WORLD WAR 2 GLIDERS ASSTD DESIGNS  2873
9   21731      RED TOADSTOOL LED NIGHT LIGHT  2868


## 재구매 고객 확인

In [13]:
cursor.execute("""
               SELECT COUNT(DISTINCT customerid) AS repurcharser_count
               FROM (
               SELECT customerid, COUNT(DISTINCT invoicedate) AS frq
               FROM sales
               WHERE customerid <> ''
               GROUP BY customerid
               HAVING  COUNT(DISTINCT invoicedate) >= 2
               ) a
               """)


df = pd.DataFrame(cursor.fetchall(), columns=['repurcharser_count'])
print(df)

   repurcharser_count
0                3058


## 특정 제품의 재구매 고객수와 구매일자 순서